In [1]:
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

data_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\search_20241106-223705_sodium+ion+battery+anode-sodium+ion+battery+cathode-sodium+ion+battery+electrode_annotated_rephrased.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

texts = []
labels_raw = []

for i in data:
    texts.append(i["text"])
    labels_raw.append(i["label_int"])

labels = []
for i in labels_raw:
    if i == 0:
        labels.append(1)
    elif i == 1:
        labels.append(0)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42
)

train_dataset = Dataset.from_dict({
    'text': train_texts,
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'text': val_texts,
    'labels': val_labels
})

raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})
print(len(texts))

400


In [2]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 320
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 80
    })
})


In [3]:
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, torch_dtype="auto")

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [6]:
import evaluate

metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  eval_strategy="epoch",
                                  num_train_epochs=10)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)


In [10]:
trainer.train()

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.4034307897090912, 'eval_accuracy': 0.8875, 'eval_runtime': 0.6455, 'eval_samples_per_second': 123.941, 'eval_steps_per_second': 15.493, 'epoch': 1.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.2209351360797882, 'eval_accuracy': 0.9625, 'eval_runtime': 0.6487, 'eval_samples_per_second': 123.326, 'eval_steps_per_second': 15.416, 'epoch': 2.0}


...

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.11851081997156143, 'eval_accuracy': 0.9875, 'eval_runtime': 0.706, 'eval_samples_per_second': 113.317, 'eval_steps_per_second': 14.165, 'epoch': 9.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.1187400221824646, 'eval_accuracy': 0.9875, 'eval_runtime': 0.6756, 'eval_samples_per_second': 118.417, 'eval_steps_per_second': 14.802, 'epoch': 10.0}
{'train_runtime': 103.3271, 'train_samples_per_second': 30.97, 'train_steps_per_second': 3.871, 'train_loss': 0.11762532234191894, 'epoch': 10.0}


TrainOutput(global_step=400, training_loss=0.11762532234191894, metrics={'train_runtime': 103.3271, 'train_samples_per_second': 30.97, 'train_steps_per_second': 3.871, 'total_flos': 841955377152000.0, 'train_loss': 0.11762532234191894, 'epoch': 10.0})

In [11]:
trainer.save_model("batterybert-cased-abstract_finetuned_400")

In [12]:
from transformers import pipeline
classifier = pipeline("text-classification", model="batterybert-cased-abstract_finetuned_400", tokenizer=tokenizer, device=0)

In [13]:
val_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai_1.json'
with open(val_path, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

label_bert = []
score_bert = []
for i in range(len(val_data["text"])):
    val_label = classifier(val_data["text"][i][:512])
    if val_label[0]["label"] == "battery":
        label_bert.append(1)
        score_bert.append(val_label[0]["score"])
    elif val_label[0]["label"] == "non-battery":
        label_bert.append(0)
        score_bert.append(val_label[0]["score"])

val_data["label_bert"] = label_bert
val_data["score_bert"] = score_bert

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [14]:
label_openai = val_data["label_openai"]
label_annotated = val_data["label_annotated"]
print(f"No\tAnno\tOpenAI\tBERT\tBERT Score")

openai_fp = 0
openai_fn = 0
bert_fp = 0
bert_fn = 0

for i in range(len(label_bert)):
    if (label_bert[i] + label_openai[i] + label_annotated[i] != 3) and (label_bert[i] + label_openai[i] + label_annotated[i] != 0):
        print(f"{i}\t{label_annotated[i]}\t{label_openai[i]}\t{label_bert[i]}\t{score_bert[i]}")
        if label_bert[i] == 1 and label_annotated[i] == 0:
            bert_fp += 1
        elif label_bert[i] == 0 and label_annotated[i] == 1:
            bert_fn += 1
        if label_openai[i] == 1 and label_annotated[i] == 0:
            openai_fp += 1
        elif label_openai[i] == 0 and label_annotated[i] == 1:
            openai_fn += 1

print(f"OpenAI False Positive: {openai_fp}")
print(f"OpenAI False Negative: {openai_fn}")
print(f"Bert False Positive: {bert_fp}")
print(f"Bert False Negative: {bert_fn}")

No	Anno	OpenAI	BERT	BERT Score
6	1	1	0	0.9998568296432495
7	0	1	0	0.9998985528945923
9	1	1	0	0.9998699426651001
11	0	1	0	0.9997000694274902
21	0	1	0	0.999904990196228
26	1	1	0	0.7653794288635254
27	1	1	0	0.9998598098754883
34	0	1	0	0.9998986721038818
36	0	1	0	0.9998993873596191
37	1	1	0	0.9998093247413635
39	1	1	0	0.9997014403343201
48	1	1	0	0.9998742341995239
53	0	1	0	0.999909520149231
72	1	1	0	0.9998168349266052
75	1	0	0	0.9999109506607056
79	0	1	0	0.9999016523361206
80	0	1	0	0.9998780488967896
83	0	0	1	0.9999241828918457
87	1	1	0	0.9998412132263184
88	0	1	0	0.9998241066932678
92	1	1	0	0.9994475245475769
95	1	1	0	0.9597983956336975
99	0	1	0	0.9998942613601685
OpenAI False Positive: 10
OpenAI False Negative: 1
Bert False Positive: 1
Bert False Negative: 12
